In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-urbanshift' 
aws_s3_dir = 'https://'+bucket_name+'.s3.eu-west-3.amazonaws.com'

# Load data

In [ ]:
# read esa land cover
esa_land_cover = ee.ImageCollection('ESA/WorldCover/v100').first()

habitatFROM = [0,10,20,30,40,50,60,80,70,90,95,100]
habitatTO = [0,1,1,1,0,0,0,0,0,1,1,1]
esa_natural_areas = esa_land_cover.remap(habitatFROM,habitatTO)

In [ ]:
# get list of urbanshift cities
boundary_georef = pd.read_csv(aws_s3_dir+'/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Extract ESA raster by feature

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = aws_s3_dir+'/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image for Land cover as GeoTIFF
    geemap.ee_export_image_to_drive(
        esa_land_cover.toByte(), # use toByte() to reduce file size 
        description = boundary_id + '-ESA-Landcover',
        folder='data', 
        scale=10, # 10, 50, or 100
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )
    
    geemap.ee_export_image_to_drive(
        esa_natural_areas.toByte(), # use toByte() to reduce file size 
        description = boundary_id + '-ESA-naturalareas',
        folder='data', 
        scale=100, # 10, 50, or 100
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [ ]:
# connect to s3
# aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)

    # read local raster
    city_file = 'data/' + boundary_id + '-ESA-Landcover100.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/land_use/esa_world_cover/v_0/'+ boundary_id + '-ESA-world_cover-2020-100m.tif',
                               ExtraArgs={'ACL':'public-read'})

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)

    # read local raster
    city_file = 'data/' + boundary_id + '-ESA-naturalareas.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/land_use/esa_world_cover/v_0/'+ boundary_id + '-ESA-world_cover-2020-natural_areas-100m.tif',
                               ExtraArgs={'ACL':'public-read'})